# Master Thesis on the Semantics of (made-up) Names

* Author: Aron Joosse
* Supervisor: Giovanni Cassani
* Institution: Tilburg University

Can take inspiration from: https://github.com/Masetto96/BA-Thesis-form-meaning-mapping/blob/master/form_meaning_mapping.ipynb

# Library Imports

In [4]:
!pip install fasttext --progress-bar off
!pip install -U spacy --progress-bar off
!python -m spacy download en_core_web_sm
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3148840 sha256=1728004d1f201d5f3c617c8067e040a6c977b5a0dc4a7ec02d5d59dfb5237188
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/







  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0

In [5]:
from google.colab import drive

# Preprocessing
import re
import os
import spacy
import pickle
import pandas as pd
from pandas import read_csv

# FastText
import fasttext
import fasttext.util

import random

# MEN and SimLex Benchmarks
from os import listdir
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity

# ElasticNet and ANN
import sklearn
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, mean_squared_error, median_absolute_error
from sklearn.linear_model import ElasticNetCV

import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
from numpy.random import seed

import time

from statistics import median

from tensorflow import keras
from tensorflow.random import set_seed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import HeNormal
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

# Data Import

In [10]:
## Getting the list of madeup names:
drive.mount("/content/drive", force_remount=True) 
ratings_csv = pd.read_csv("drive/MyDrive/Thesis/Data/giovanni_email_data/avgRatings_annotated.csv",
                          usecols = ["name", "name_type"])                      ## Importing the names and name types

ratings_csv.head(10)

madeup_names = []

for i in ratings_csv.index:                                                     ## Only choosing madeup names so I can filter them out of the FT vocab
  if ratings_csv["name_type"][i] == "madeup":
    madeup_names.append(str(ratings_csv["name"][i]))

madeup_names_lower = list(map(lambda x: x.lower(), madeup_names))               ## Lowercasting the names since my entire vocab will be lowercast

print(madeup_names[:5])
print(len(madeup_names))
print(madeup_names_lower[:5])
print(len(madeup_names_lower))

Mounted at /content/drive
['Alastor', 'Alecto', 'Amabala', 'Araminta', 'Arcturus']
60
['alastor', 'alecto', 'amabala', 'araminta', 'arcturus']
60


In [7]:
path = "drive/My Drive/Thesis/Data/CoCA/Text/"                                  ## These are the paths to easily export/import my dicts, txts, models, and pickles
dict_path = "drive/My Drive/Thesis/Data/CoCA/dict_pickles/"
unclean_path = path + "texts_combined/all_texts_combined.txt"
model_path = "drive/My Drive/Thesis/Data/CoCA/models/"
pickle_path = "drive/MyDrive/Thesis/Data/fastText and others/"
norms_path = "drive/My Drive/Thesis/Data/Norms/"
csv_path = "drive/My Drive/Thesis/Data/CSV/"

## COCA

In [ ]:
unclean_corpus = open(unclean_path).read()                                      ## Importing the entire coca

In [ ]:
print(len(unclean_corpus))                                                      ## Showing the length and first 100 characters of the coca
print(unclean_corpus[:100])

2977527143
@@4170367 Headnote # A puzzle has long pervaded the criminal law : why are two offenders who commit 


## Names

In [ ]:
### Read CSV File and Delete Unimportant Columns (i.e., everything that isn't the name, name type, rating, or the author's choice)

### This is input for the FT model, which itself is the input for the ElasticNet and ANN regressions

names_ratings = read_csv("drive/MyDrive/Thesis/Data/giovanni_email_data/avgRatings_annotated.csv")

#print(names_ratings.head())

print(names_ratings['rating.mean_age'].notna().sum())                           ## Choosing only those rows where all columns are not NA
print(names_ratings['rating.mean_gender'].notna().sum())
print(names_ratings['rating.mean_valence'].notna().sum())

df_age = names_ratings.loc[names_ratings['rating.mean_age'].notna(), ['name', 'rating.mean_age', 'age', 'name_type']]   ## Choosing the relevant columns
print(df_age.head(), len(df_age))

df_gender = names_ratings.loc[names_ratings['rating.mean_gender'].notna(), ['name', 'rating.mean_gender', 'gender', 'name_type']]
print(df_gender.head(), len(df_gender))

df_polarity = names_ratings.loc[names_ratings['rating.mean_valence'].notna(), ['name', 'rating.mean_valence', 'polarity', 'name_type']]
print(df_polarity.head(), len(df_polarity))

119
179
63
       name  rating.mean_age    age name_type
0  Adelaide        -0.617647    old      real
2  Alasdair        18.709677  young      real
3   Alastor        13.812500    old    madeup
4    Alecto         3.593750    old    madeup
5     Alice       -13.969697  young      real 119
       name  rating.mean_gender  gender name_type
0  Adelaide           45.727273  female      real
1   Adelina           47.771429  female      real
2  Alasdair          -35.657143    male      real
3   Alastor          -38.833333    male    madeup
4    Alecto          -35.722222  female    madeup 179
        name  rating.mean_valence polarity name_type
1    Adelina            31.621622      bad      real
7    Amabala             5.935484     good    madeup
8      Apple            32.444444     good   talking
11  Arcturus           -11.166667     good    madeup
13   Arobynn             7.645161      bad    madeup 63


# Preprocessing


## Cleaning Corpus

In [ ]:
## Loading the English spacy pipeline and removing stopwords (since we are interested in gender bias, it's best to leave these words in)

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 10000000000

nlp.Defaults.stop_words.remove('him')
nlp.Defaults.stop_words.remove('her')
nlp.Defaults.stop_words.remove('hers')
nlp.Defaults.stop_words.remove('his')
nlp.Defaults.stop_words.remove('he')
nlp.Defaults.stop_words.remove('she')
nlp.Defaults.stop_words.remove('himself')
nlp.Defaults.stop_words.remove('herself')

In [ ]:
def clean_corpus_sentenced(data, corpus_dict, index):
  ## Input: 
  # - Data = A (very) large string of corpus text
  # - Corpus_dict = a dictionary to store individual sentences in
  # - Index = the last index from the previous batch

  ## Process: 
  # Remove all unwanted tokens, and store individual sentences in the dictionary

  ## Output: 
  # - The dictionary of preprocessed sentences
  # - The last sentence index, for the next batch to continue with (so that the order of the sentences is kept)

  # Tokenization
  with nlp.select_pipes(disable=["lemmatizer", "tok2vec", "tagger", "parser"]):
    nlp.enable_pipe("senter")                                                   ## Helps with better segmenting into sentences
    doc = nlp(data)

  sentence = ""                                                                 ## Initialize an empty sentence

  for token in doc:
    if token.is_sent_start is True:                                             ## If token is the star of the sentence, add the previous sentence to the dictionary
      if sentence == "":                                                        ## and create a new, clean sentence
        continue
      else:
        corpus_dict[index] = sentence
        sentence = ""
        index += 1
    
    if token.is_upper is True:                                                  ## Remove all full-caps words
      continue
    elif token.is_stop is True:                                                 ## Remove all stopwords
      continue
    elif str(token).lower() in madeup_names_lower:                              ## Remove all words that are in my list of made-up names
      continue
    elif token.is_alpha:                                                        ## If the token has passed all previous tests, and it consists only of alphabetic
      sentence += str(token).lower() + " "                                      ## characters, lowercast it and add an extra space to the end; continue to the next
                                                                                ## token
  return corpus_dict, index

In [ ]:
def corpus_dict_maker(data, start, end, index):
  ## Input: 
  # - Data = The entire uncleaned corpus
  # - Start = The character index to indicate the start of the current batch to preprocess
  # - End = The character index to indicate the end of the current batch to preprocess
  # - Index = The sentence index from the previous batch, to keep track of the number and order of the sentences

  ## Process: 
  # Preprocess the corpus in batches, since there was not enough RAM to preprocess the entire corpus at once
  # So, for every batch, all the characters between the start index and the end index are fed into the clean_corpus_sentenced() function
  # and then this dictionary of sentences is saved as a pickle to Google drive

  ## Output: 
  # Nothing; except that the sentence index is printed, which is used as input for the next batch (this was printed, so that it couldn't be lost if the 
  # runtime would disconnect (which it sadly did very often))

  drive.mount("/content/drive", force_remount=True)                             ## Connect to google drive
  
  corpus_dict = {}                                                              ## Create empty dictionary

  prev_i = (start-2)*1000000                                                    ## Start with preprocessing the two million characters before the previous index
                                                                                ## since the next range only preprocesses up to but not including the 'end' index

  for i in range(start, end, 2):                                                ## In batches of 2 million characters, feed the batch to clean_corpus_sentenced()
    print(i)                                                                   
    i *= 1000000
    corpus_dict, index = clean_corpus_sentenced(data[prev_i:i],
                                                corpus_dict,
                                                index)
    prev_i = i
  
  if prev_i == 2976000000:                                                      ## Hardcoded; if we get near the end of the corpus, don't preprocess in a batch of
    corpus_dict, index = clean_corpus_sentenced(data[prev_i:],                  ## 2 million characters (we would get an out of range error), but rather just 
                                                corpus_dict,                    ## preprocess the remaining characters, however many that may be
                                                index)

  print(index)

  pickle_out = open(dict_path + "corpus_dict_until_" + str(end) + ".pickle", "wb")  ## Save the dictionary as a pickle
  pickle.dump(corpus_dict, pickle_out)
  pickle_out.close()

  drive.flush_and_unmount()                                                     ## Flush the pickle to my drive
  print('All changes made in this colab session should now be visible in Drive.')



#### All of the individual batches:

In [ ]:
## doing it in batches to 
## (1) make it possible in terms of time and the Google afk-checker captcha pop-up, and 
## (2) to not blow out the RAM and have it break down

#corpus_dict_maker(unclean_corpus, 2, 500, 0)                   
#corpus_dict_maker(unclean_corpus, 500, 640, 3217086)           
#corpus_dict_maker(unclean_corpus, 640, 760, 4232218)           
#corpus_dict_maker(unclean_corpus, 760, 800, 5439287)           
#corpus_dict_maker(unclean_corpus, 800, 900, 5888161)
#corpus_dict_maker(unclean_corpus, 900, 980, 7020129)
#corpus_dict_maker(unclean_corpus, 980, 1150, 7891661)
#corpus_dict_maker(unclean_corpus, 1150, 1200, 9903820) 
#corpus_dict_maker(unclean_corpus, 1200, 1204, 10502592)

In [ ]:
######### This block is separate because it kept crashing if I did it otherwise:

#drive.mount("/content/drive", force_remount=True) 
#corpus_dict = {}

#prev_i = (1202)*1000000
#i = 1203*1000000
#corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:i],
#                                            corpus_dict,
#                                            10547544)
#prev_i = i

#print(index)

#pickle_out = open(path + "corpus_dict_until_" + str(1203) + "_post_1204" + ".pickle", "wb")
#pickle.dump(corpus_dict, pickle_out)
#pickle_out.close()

#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

################################################################################

######### This block is separate because it kept crashing if I did it otherwise:

#drive.mount("/content/drive", force_remount=True) 
#corpus_dict = {}

#prev_i = (1203)*1000000
#i = 1204*1000000
#corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:i],
#                                            corpus_dict,
#                                            index + 1)
#prev_i = i

#print(index)

#pickle_out = open(path + "corpus_dict_until_" + str(1204) + "_post_1204" + ".pickle", "wb")
#pickle.dump(corpus_dict, pickle_out)
#pickle_out.close()

#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

################################################################################

######### This block is separate because it kept crashing if I did it otherwise:

#drive.mount("/content/drive", force_remount=True) 
#corpus_dict = {}

#prev_i = (1204)*1000000
#i = 1205*1000000
#corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:i],
#                                            corpus_dict,
#                                            10580543)
#prev_i = i

#print(index)

#pickle_out = open(path + "corpus_dict_until_" + str(1205) + "_post_1204" + ".pickle", "wb")
#pickle.dump(corpus_dict, pickle_out)
#pickle_out.close()

#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

################################################################################

######### This block is separate because it kept crashing if I did it otherwise:

#drive.mount("/content/drive", force_remount=True) 
#corpus_dict = {}

#prev_i = (1205)*1000000
#i = 1206*1000000
#corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:i],
#                                            corpus_dict,
#                                            index + 1)
#prev_i = i

#print(index)

#pickle_out = open(path + "corpus_dict_until_" + str(1206) + "_post_1204" + ".pickle", "wb")
#pickle.dump(corpus_dict, pickle_out)
#pickle_out.close()

#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

################################################################################

In [ ]:
#corpus_dict_maker(unclean_corpus, 1208, 1300, 10611881)
#corpus_dict_maker(unclean_corpus, 1300, 1500, 11486498)
#corpus_dict_maker(unclean_corpus, 1500, 1750, 13172710)
#corpus_dict_maker(unclean_corpus, 1750, 1846, 15332847)
#corpus_dict_maker(unclean_corpus, 1846, 1848, 16123425)
#corpus_dict_maker(unclean_corpus, 1848, 1850, 16147433)
#corpus_dict_maker(unclean_corpus, 1850, 1900, 16172965)
#corpus_dict_maker(unclean_corpus, 1900, 1968, 16855832)
#corpus_dict_maker(unclean_corpus, 1968, 1970, 17790964)
#corpus_dict_maker(unclean_corpus, 1970, 2000, 17819821)
#corpus_dict_maker(unclean_corpus, 2000, 2022, 18244076)
#corpus_dict_maker(unclean_corpus, 2022, 2024, 18536113)
#corpus_dict_maker(unclean_corpus, 2024, 2026, 18558956)
#corpus_dict_maker(unclean_corpus, 2026, 2068, 18583534)
#corpus_dict_maker(unclean_corpus, 2068, 2070, 19154020)
#corpus_dict_maker(unclean_corpus, 2070, 2100, 19179335)
#corpus_dict_maker(unclean_corpus, 2100, 2166, 19598984)
#corpus_dict_maker(unclean_corpus, 2166, 2168, 20488725)
#corpus_dict_maker(unclean_corpus, 2168, 2188, 20524278)

In [ ]:
######### This block is separate because it kept crashing if I did it otherwise:

#drive.mount("/content/drive", force_remount=True) 
#corpus_dict = {}

#prev_i = (2186)*1000000
#i = 2187*1000000
#corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:i],
#                                            corpus_dict,
#                                            20779021)
#prev_i = i

#print(index)

#pickle_out = open(path + "corpus_dict_until_" + str(2187) + "_post_2188" + ".pickle", "wb")
#pickle.dump(corpus_dict, pickle_out)
#pickle_out.close()

#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

################################################################################

######### This block is separate because it kept crashing if I did it otherwise:

#drive.mount("/content/drive", force_remount=True) 
#corpus_dict = {}

#prev_i = (2187)*1000000
#i = 2188*1000000
#corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:i],
#                                            corpus_dict,
#                                            index + 1)
#prev_i = i

#print(index)

#pickle_out = open(path + "corpus_dict_until_" + str(2188) + "_post_2188" + ".pickle", "wb")
#pickle.dump(corpus_dict, pickle_out)
#pickle_out.close()

#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

################################################################################

In [ ]:
#corpus_dict_maker(unclean_corpus, 2190, 2200, 20812123)
#corpus_dict_maker(unclean_corpus, 2200, 2310, 20945992)
#corpus_dict_maker(unclean_corpus, 2310, 2312, 22397914)
#corpus_dict_maker(unclean_corpus, 2312, 2400, 22424124)
#corpus_dict_maker(unclean_corpus, 2400, 2600, 23465826)
#corpus_dict_maker(unclean_corpus, 2600, 2800, 25199888)
#corpus_dict_maker(unclean_corpus, 2800, 2977, 26938737)

In [ ]:
## This block of code opens all of the corpus dict pickles, and combines them together
## into one big dictionary: corpus_dict_complete

file_number = 1
file_list = []
for file_name in os.listdir(dict_path):                                         ## Locate all dicts in the folder
  with open(dict_path + str(file_name), 'rb') as f:
    exec("dict_" + str(file_number) + " = " + "pickle.load(f)")
    file_list.append("dict_" + str(file_number))    
    file_number += 1
  
corpus_dict_complete = {}
for file_name in file_list:                                                     
  corpus_dict_complete = {**corpus_dict_complete, **globals()[file_name]}       ## Add the contents of the dicts to dict_complete
  del globals()[file_name]

#print(len(corpus_dict_complete))
#del file_number
#del file_list

#pickle_out = open(dict_path + "corpus_dict_complete.pickle", "wb")             ## Create a new pickle
#pickle.dump(corpus_dict_complete, pickle_out)
#del corpus_dict_complete
#pickle_out.close()

#drive.flush_and_unmount()                                                      ## Flush the pickle to my drive
#print('All changes made in this colab session should now be visible in Drive.')

In [ ]:
## This block of code opens the corpus_dict_complete pickle, and loops through the keys by index (from lowest to highest).
## So, we loop through every sentence, in order. These are stored in two different .txt files, one where the sentence structure is remained 
## (i.e., between every sentence, we add a newline character), and one that's unsentenced (i.e., no newline character between sentences).

with open(dict_path + "corpus_dict_complete.pickle", "rb") as d:                ## Open corpus_dict_complete
  corpus_dict_complete = pickle.load(d)

  with open(path + "cleaned_sentenced_corpus_complete.txt", "w") as f:          ## Open sentenced corpus .txt file
    for key in sorted(corpus_dict_complete):
      if len(str(corpus_dict_complete[key]).split()) < 2:                       ## Remove sentences with only 1 word (since there's no 'context' in that case)
        continue
      else:
        if str(corpus_dict_complete[key])[:2] in ["m ", "p ", "s "]:            ## I have to add this, because based on manual inspection, a significant amount of 
          f.write(str(corpus_dict_complete[key])[2:] + "\n")                    ## sentences start with just a "p", "m", or "s"
        else:
          f.write(str(corpus_dict_complete[key]) + "\n")

  with open(path + "cleaned_unsentenced_corpus_complete.txt", "w") as f2:       ## Open unsentenced corpus .txt file
    for key in sorted(corpus_dict_complete): 
      if len(str(corpus_dict_complete[key])) < 4:                               ## Remove sentences with less than 4 characers, since based on visual inspection, I
        continue                                                                ## saw that such sentences are mostly nonsense (i.e., not real words)
      else:
        if str(corpus_dict_complete[key])[:2] in ["m ", "p ", "s "]:
          f2.write(str(corpus_dict_complete[key])[2:])
        else:
          f2.write(str(corpus_dict_complete[key]))
  
drive.flush_and_unmount()                                                       ## Flush to drive
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## Training fastText and Validating on MEN and SimLex999 + Training W2V

### fastText

In [ ]:
# def fasttext_tuner(data_type):                                                ## I have chosen Skipgram, I won't play around with epochs or learning rate
#   for dimensionality in [100, 300]:                                           ## I'm first checking data type and dimensionality, and choosing the most promising combination.
#     for window_size in [2, 3, 4, 5, 6, 7]:                                    ## Then I'll check window size and choose the 3 most promising ones
#       for min_n in [1, 2, 3]:                                                 ## Then I want to check min_n 2 and 1 to see whether adding n-gram size of 1 makes any sense
#         for max_n in [5, 6, 7]:                                               ## Lastly, I will iterate through the 3 * 2 * 3 most promising models, finally choosing the best one and then double checking that with
#           model = fasttext.train_unsupervised(input = path + "cleaned_" +     ## the 100/300 sentenced/unsentenced options, just to be sure! In total, this means training 4 + 5 + ~16 + 3 = ~30 models instead of 216
#                                               data_type + 
#                                               "_corpus_complete.txt",
#                                               model = "skipgram",
#                                               dim = dimensionality, 
#                                               ws = window_size, 
#                                               minn = min_n,
#                                               maxn = max_n)
          
#           model.save_model(model_path + data_type + "_dim" + str(dimensionality) + 
#                             "_ws" + str(window_size) + "_minn" + str(min_n) + 
#                             "_maxn" + str(max_n) + ".bin")
          
#           print(data_type + "_dim" + str(dimensionality) + "_ws" + str(window_size) + 
#                 "_minn" + str(min_n) + "_maxn" + str(max_n))

#           del model

#   drive.flush_and_unmount()
#   print('All changes made in this colab session should now be visible in Drive.')

In [ ]:
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')
ft.save_model(model_path + "pretrained_model.bin")

In [ ]:
def fasttext_tuner(data_type, dimensionality = 300, window_size = 5, min_n = 3, max_n = 6):
  ## Input: 
  # - Data Type: Sentenced or unsentenced, to load the corpus type to train model
  # - Dimensionality: List of dimensionalities
  # - Window size: List of window sizes 
  # - Min_n: List of minimum n-gram sizes 
  # - Max-n: List of maximum n-gram sizes 

  ## Process: 
  # For every combination of input variables, a fastText model is trained and saved

  ## Output:
  # Nothing; fastText model is trained and saved

  for d in dimensionality:                                           
    for ws in window_size:                                    
      for minn in min_n:                                                 
        for maxn in max_n:
          drive.mount("/content/drive", force_remount=True)                     ## Mount drive

          model = fasttext.train_unsupervised(input = path + "cleaned_" + data_type + "_corpus_complete_without_p_s_m.txt",   ## Train model
                                              model = "skipgram", dim = d, ws = ws, minn = minn, maxn = maxn)

          model.save_model(model_path + data_type + "_dim" + str(d) + "_ws" + str(ws) + "_minn" + str(minn) + "_maxn" + str(maxn) + ".bin")   ## Save model

          print(data_type + "_dim" + str(d) + "_ws" + str(ws) + "_minn" + str(minn) + "_maxn" + str(maxn))

          drive.flush_and_unmount()                                             ## Flush model to drive
          print('All changes made in this colab session should now be visible in Drive.')

          del model

In [ ]:
## Trying out some variations of fastText with Data = sentenced & Dimensionality = 100

fasttext_tuner("sentenced", [100], [2], [2], [3])
fasttext_tuner("sentenced", [100], [2, 3, 4, 5, 6, 7], [2], [5, 6, 7])
fasttext_tuner("sentenced", [100], [5], [3], [6])
fasttext_tuner("sentenced", [100], [5], [1], [6])

## Trying out some variations of fastText with Data = sentenced & Dimensionality = 300

fasttext_tuner("sentenced", [300], [3, 4], [2], [5, 6, 7])
fasttext_tuner("sentenced", [300], [5], [2], [5, 6])
fasttext_tuner("sentenced", [300], [5], [3], [6])
fasttext_tuner("sentenced", [300], [3], [3], [5])
fasttext_tuner("sentenced", [300], [3], [2], [4])
fasttext_tuner("sentenced", [300], [2, 6, 7], [2], [5])

## Trying out some variations of fastText with Data = unsentenced 

fasttext_tuner("unsentenced", [300], [5], [3], [6])
fasttext_tuner("unsentenced", [100], [5], [3], [6])

In [ ]:
## Doing it one last time, this time with seeds + an n-gramless model

random.seed(17042020)

fasttext_tuner('sentenced', [300], [2], [2], [5])

random.seed(17042020)

fasttext_tuner('sentenced', [300], [2], [0], [0])

Mounted at /content/drive


### MEN and SimLex Benchmarks

In [ ]:
def compute_similarity(model, word1, word2):
  # Input:
  # - Model = the model that is used to get embeddings for word1 and word2
  # - Word1 = the first word
  # - Word2 = the second word

  # Process: 
  # Given the embeddings for word1 and word2 derived from the model, calculate
  # the cosine similarity between these words

  # Output:
  # The cosine similarity between word1 and word2 given the current model

    e1 = model[word1]
    e2 = model[word2]
    s = cosine_similarity(e1.reshape(1, -1), e2.reshape(1, -1))
    return s[0][0]

In [ ]:
def compute_correlation_men(norms, model):
  # Input:
  # - Norms = The MEN similarity scores for word1 and word2
  # - Model = the model that will be scored

  # Process: 
  # For every model, note down the true similarity score (as dictated by the MEN
  # benchmark) and then estimate the similarity for the current model using the 
  # compute_similarity() function

  # Output:
  # The Spearman R between the true and estimated similarities for the current
  # model.

        true_similarities = []
        estimated_similarities = []
        for _, row in norms.iterrows():
            s = compute_similarity(model, row['w1'], row['w2'])

            estimated_similarities.append(s)
            true_similarities.append(row['sim'])
        
        return spearmanr(true_similarities, estimated_similarities)[0]

In [ ]:
def compute_correlation_simlex(norms, model):
  # Input:
  # - Norms = The SimLex similarity scores for word1 and word2
  # - Model = the model that will be scored

  # Process: 
  # For every model, note down the true similarity score (as dictated by the SimLex
  # benchmark) and then estimate the similarity for the current model using the 
  # compute_similarity() function

  # Output:
  # The Spearman R between the true and estimated similarities for the current
  # model.

        true_similarities = []
        estimated_similarities = []
        for _, row in norms.iterrows():
            s = compute_similarity(model, row['word1'], row['word2'])

            estimated_similarities.append(s)
            true_similarities.append(row['SimLex999'])
        
        return spearmanr(true_similarities, estimated_similarities)[0]

In [ ]:
men_norms = pd.read_csv(norms_path + 'norms.dev.csv', sep = ' ')                # Read the MEN benchmark CSV file

combi_score = {}
men_dict = {}

for model in os.listdir(model_path):                                            # For every saved model, calculate the scores on the MEN benchmarks
  men_dict[str(model)] = compute_correlation_men(men_norms, model)
  combi_score[str(model)] = compute_correlation_men(men_norms, model)

men_dict['pretrained_model'] = compute_correlation_men(men_norms, ft)           # Calculating the scores for the baseline (pretrained) model
combi_score['pretrained_model'] = compute_correlation_men(men_norms, ft)

In [ ]:
simlex_norms = pd.read_csv(norms_path + 'SimLex-999.txt', sep = '\t')           # Read the SimLex benchmark text file

simlex_dict = {}

for model in os.listdir(model_path):                                            # For every saved model, calculate the scores on the SimLex benchmarks
  simlex_dict[str(model)] = compute_correlation_simlex(simlex_norms, model)
  combi_score[str(model)] = compute_correlation_simlex(simlex_norms, model)

simlex_dict['pretrained_model')] = compute_correlation_simlex(simlex_norms, ft) # Calculating the scores for the baseline (pretrained) model
combi_score['pretrained_model'] = compute_correlation_simlex(simlex_norms, ft)

In [ ]:
pickle_men = open(pickle_path + "men_scores.pickle", "wb")                       # Save the MEN scores as a pickle
pickle.dump(men_dict, pickle_men)
pickle_men.close()

pickle_simlex = open(pickle_path + "simlex_scores.pickle", "wb")                 # Save the SimLex scores as a pickle
pickle.dump(simlex_dict, pickle_simlex)
pickle_simlex.close()

pickle_combi = open(pickle_path + "combi_scores.pickle", "wb")                   # Save the combined scores as a pickle
pickle.dump(combi_score, pickle_combi)
pickle_combi.close()

In [ ]:
for i in sorted(men_dict, key = men_dict.get, reverse = True):
    print(i, men_dict[i])

In [ ]:
for i in sorted(simlex_dict, key = simlex_dict.get, reverse = True):
    print(i, simlex_dict[i])

In [ ]:
for i in sorted(combi_score, key = combi_score.get, reverse = True):
    print(i, combi_score[i])

### Final Model

In [11]:
model = fasttext.load_model(model_path + 'sentenced_dim300_ws2_minn2_maxn5.bin')
model_subwordless = fasttext.load_model(model_path + 'sentenced_dim300_ws2_minn0_maxn0.bin')

## ElasticNet and Neural Network Regressions

In [1]:
def fasttext_xifyer(input_data):
  # Input:
  # - input_data = dataframe containing the names of the characters

  # Process:
  # Convert the character names to embeddings using FT

  # Output:
  # - df_output = a dataframe of word embeddings; one embedding per character name

  df_output = np.zeros((len(input_data), 300))                                  # Create an array of length = number of input rows & width = 300 (because of ft)

  i = 0

  for row in input_data.iterrows():                                             # For every row in the input data
    index = row[0]
    name = row[1][0].lower()                                                    # Lowercast the name
    df_output[i] = model[name]                                                  # Compute the fastText score of the name, and add it to the output array
    i += 1

  return df_output

In [2]:
def fasttext_xifyer_ngram(input_data):
  # Input & Output are exactly the same as fasttext_xifyer

  # Process:
  # Similar to fasttext_xifyer, however, if the name has a 'surface' form that 
  # is already present in the model vocabulary, then we only use the embeddings
  # for the subwords. I.e., as compared to fasttext_xifyer, this function does 
  # NOT consider the 'surface' form embedding for the character names

  df_output = np.zeros((len(input_data), 300))                                  # Create an array of length = number of input rows & width = 300 (because of ft)

  c = 0

  for row in input_data.iterrows():                                             # For every row in the input data
    name = row[1][0].lower()
    if name == model.get_subwords(name)[0][0]:                                  # If the surface form of the name is present in the model vocabulary                                                                        
      wordarray = np.zeros(model[name].shape)
      subword_counter = 0
      for subword_hash in model.get_subwords(name)[1][1:]:
        wordarray += model.get_input_vector(subword_hash)
        subword_counter += 1
      
      df_output[c] = wordarray / subword_counter              
                                    
    else:
      df_output[c] = model[name]                                                # Else (if no surface name is present in the model vocabulary), just take the embedding of the name as is
    c += 1

  return df_output

In [3]:
def fasttext_xifyer_lexical(input_data):
  # Input & Output are exactly the same as fasttext_xifyer

  # Process:
  # Similar to fasttext_xifyer, however, if the name has a 'surface' form that 
  # is already present in the model vocabulary, then we only use the embeddings
  # for the subwords. I.e., as compared to fasttext_xifyer, this function does 
  # NOT consider the 'surface' form embedding for the character names

  df_output = np.zeros((len(input_data), 300))                                  # Create an array of length = number of input rows & width = 300 (because of ft)

  c = 0

  for row in input_data.iterrows():                                             # For every row in the input data
    name = row[1][0].lower()
    if name == model.get_subwords(name)[0][0]:                                  # If the surface form of the name is present in the model vocabulary                                                                        
      #df_output[c] = model.get_input_vector(model.get_subwords(name)[1][0])                    
      df_output[c] = model_subwordless[name]                              
    else:
      continue
      #df_output[c] = mean_vector_subwordless                                    # Else (if no surface name is present in the model vocabulary), just take the mean vector
    c += 1

  return df_output

In [15]:
mean_vector = np.zeros((len(model.words), 300))                                 # Create a mean vector array of length = amount of words in the model vocab, and width = 300

i = 0

for word in model.words:                          
  mean_vector[i] = model.get_word_vector(word)                                  # For every word in the vocab, get it's word embedding vector and add it to the mean_vector array
  i += 1

print(len(mean_vector))
print(np.var(mean_vector, axis = 0))
mean_vector = np.mean(mean_vector, axis = 0)                                    # Take the mean of the array so that we get a mean_vector of length = 1, and width = 300

288636
[0.02983145 0.03451331 0.03508377 0.03308105 0.03175245 0.03168462
 0.02664619 0.03083584 0.03610629 0.02618621 0.03060608 0.03118188
 0.03008398 0.02962754 0.03095815 0.03168962 0.03058541 0.03089621
 0.03278704 0.02896047 0.03256722 0.03102792 0.02958851 0.02916435
 0.02705777 0.02733735 0.03294334 0.03009464 0.03103532 0.03033244
 0.02443951 0.02806274 0.02946075 0.03079357 0.02694207 0.02947974
 0.02916249 0.02906072 0.02854735 0.03011373 0.03179324 0.03302893
 0.0402131  0.02850775 0.02794405 0.0343181  0.03214487 0.04232577
 0.03521807 0.02888429 0.02880519 0.03060809 0.02870999 0.02828811
 0.02760074 0.02627071 0.02750845 0.03336155 0.02773434 0.03161439
 0.02697407 0.02986084 0.03375155 0.02812287 0.03417457 0.02939953
 0.02756125 0.03178698 0.02917939 0.02909285 0.03613458 0.02575054
 0.04111918 0.02836128 0.02960388 0.03104862 0.02639434 0.03038574
 0.03025171 0.03179222 0.03266656 0.02760389 0.03177202 0.02792622
 0.03132294 0.03375284 0.03209847 0.03324032 0.03152891

In [13]:
mean_vector_subwordless = np.zeros((len(model_subwordless.words), 300))         # Create a mean vector array of length = amount of words in the model vocab, and width = 300

i = 0

for word in model_subwordless.words:                          
  mean_vector_subwordless[i] = model_subwordless.get_word_vector(word)          # For every word in the vocab, get it's word embedding vector and add it to the mean_vector array
  i += 1

print(np.var(mean_vector_subwordless, axis = 0))
mean_vector_subwordless = np.mean(mean_vector_subwordless, axis = 0)            # Take the mean of the array so that we get a mean_vector of length = 1, and width = 300

[0.02215305 0.01842975 0.02453903 0.03417017 0.0226679  0.02957149
 0.02515212 0.02312255 0.02457385 0.0229695  0.0255001  0.01953455
 0.02153591 0.02144246 0.02917741 0.02650796 0.02654819 0.02836484
 0.03101649 0.02782    0.0253935  0.0228241  0.02119084 0.02328785
 0.02318165 0.02343835 0.0273614  0.02534298 0.0237404  0.02432296
 0.02344552 0.02026982 0.02516113 0.02935723 0.02569875 0.02277913
 0.02807889 0.02303279 0.02270223 0.02610916 0.02367382 0.02378246
 0.02066414 0.02686275 0.02707682 0.02103598 0.02534474 0.025892
 0.023999   0.02302867 0.02283955 0.02732134 0.0243046  0.02187653
 0.02367846 0.02205308 0.02242267 0.02215969 0.02782074 0.02909465
 0.03064635 0.02514587 0.02983635 0.02131716 0.02783101 0.02267712
 0.02432576 0.02346172 0.02271207 0.02442423 0.02447321 0.02218473
 0.02091241 0.02382913 0.0255532  0.02105344 0.02337568 0.02053997
 0.02435472 0.02260632 0.02261744 0.02948487 0.0236003  0.02406779
 0.02710708 0.02457014 0.02043468 0.02706705 0.02121432 0.021098

In [ ]:
def model_evaluator(trained_model, x_train, x_test, y_train, y_test, pred_dict, 
                    test_names, test_name_types, lexical = None):
  y_pred = trained_model.predict(x_test)
  
  mae_test = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  
  ##### MAE per name type ######################################################

  type_dict = {}
  type_dict_median = {'madeup' : [], 'real' : [], 'talking' : []}
  type_counter = {}
  for n, i, j in zip(test_name_types, y_test, x_test):                          # For every name type (i.e., real, talking, and madeup)
    if n in type_dict.keys():
      type_dict[n] = type_dict[n] + abs(i - trained_model.predict(j.reshape(1, -1)))   # Append the MAE for every name given that type (so that you get a sum of MAEs; one for each name)
      type_dict_median[n].append(abs(i - trained_model.predict(j.reshape(1, -1))))     # Append the absolute error for every name given that type
      type_counter[n] = type_counter[n] + 1                                     # And count the number of names given that type
    else:
      type_dict[n] = abs(i - trained_model.predict(j.reshape(1, -1)))
      type_dict_median[n].append(abs(i - trained_model.predict(j.reshape(1, -1))))
      type_counter[n] = 1

  for i in type_dict.keys():
    globals()[f"mae_{i}"] = float(type_dict[i])/float(type_counter[i])          # Calculate the average MAE per name type: (sum of MAEs for name type / name counter for name type)

  if 'madeup' not in type_dict.keys():
    globals()[f"mae_madeup"] = None

  ##### Mean Only ##############################################################

  if lexical is True:
    mean_vec_array = np.full((len(x_test), 300), mean_vector_subwordless)
  else:
    mean_vec_array = np.full((len(x_test), 300), mean_vector)                     # Create a mean vector array with length = test_set_length, and width = 300

  mean_vec_mae_test = mean_absolute_error(y_test, trained_model.predict(mean_vec_array))# Retrieve the MAE for the mean vector array
  
  mean_vec_r2 = r2_score(y_test, trained_model.predict(mean_vec_array))         # Retrieve the R2 for the mean_vector array

  return mae_test, r2, mae_madeup, mae_real, mae_talking, mean_vec_mae_test, mean_vec_r2

In [ ]:
column_list_pred = ['Name', 'NameType', 'TrueRating', 'CombinedRating',          # Initialize a list of column names for the prediction CSVs that will be created later
                    'CombinedMeanVecRating', 'NgramRating',
                    'NgramMeanVecRating', 'LexicalRating', 
                    'LexicalMeanVecRating']

### ElasticNet

##### Functions

In [ ]:
def elasticnetifyer(x_train, y_train, x_test, y_test, pred_dict, test_names, test_name_types, lexical = None):
  # Input:
  # - x_train = array of embeddings used to train the model
  # - y_train = array of ratings used to train the model
  # - x_test = array of embeddings used to test the model
  # - y_test = array of ratings used to test the model
  # - pred_dict = a dictionary that will be filled with predictions per name
  # - test_names = dataframe containing the full names (i.e., not the embeddings) 
  # - test_name_types = dataframe containing the name type (real, madeup, talking)

  # Process:
  # Given x_train and y_train, train the ElasticNet CV model. When it converges,
  # calculate the MSE, MAE, and R2 for a bunch of different settings (test set,
  # name types, mean_vector-only model, mean_vector_or_surface_form model)

  # Output:
  # --> Too big to all individually explain here. The ouput contains the alpha, 
  # l1_ratio, n_iters, intercept, and MSE of the trained model.
  # Furthermore, it outputs the MAE, MSE, and R2 of the test set, as well as the
  # MAE for the three different name types, the MAE for a mean_vector_or_surface
  # _form-model, and the MAE and R2 for a mean_vector-only model.  

  regr = ElasticNetCV(l1_ratio = [0.01, 0.05, .1, 0.2, .5, .7, .9, .95, .99, 1],
                      n_alphas = 250,
                      max_iter = 10000,
                      cv = len(x_train),
                      selection = 'random', 
                      random_state=17042020,)                                   # Set the hyperparameters of the ElasticNet CV

  regr.fit(x_train, y_train)                                                    # Fit the model on the train set

  alpha = regr.alpha_                                                           # Retrieve the alpha
  l1_ratio = regr.l1_ratio_                                                     # Retrieve the L1-ratio
  n_iters = regr.n_iter_                                                        # Retrieve the number of iterations needed to train the model
  intercept = regr.intercept_                                                   # Retrieve the intercept of the model

  mae_test, r2, mae_madeup, mae_real, mae_talking, mean_vec_mae_test, mean_vec_r2 \
  = model_evaluator(regr, x_train, x_test, y_train, y_test, 
                    pred_dict, test_names, test_name_types, lexical)             # Evaluate the model using the evaluator function

  return alpha, l1_ratio, n_iters, intercept, mae_test, r2, \
  mae_madeup, mae_real, mae_talking, mean_vec_mae_test, mean_vec_r2

In [ ]:
def nested_cross_validator(df, rating, dictionary):
  # Input:
  # - df = a dataframe with the name, name_type, and rating for the dimension at  
  # hand (i.e., age, gender, or polarity)
  # - rating = a string indicating what rating to extract from the df
  # - dictionary = an empty dictionary to store all of the variables from 
  # elasticnetifyer() in

  # Process:
  # Given the df, get 5 train/test splits, and per fold, train a LOOCV ElasticNet
  # model using elasticnetifyer() for both the combined and ngram data. 
  # Then, average the scores retrieved on the different folds, and print it.

  # Output: 
  # - pred_dict_combined: y_true and y_pred (for several conditions) per name (combined)
  # - pred_dict_ngram: y_true and y_pred (for several conditions) per name (ngram)
  # - pred_dict_lexical: y_true and y_pred (for several conditions) per name (lexical)
  
  pred_dict_combined = {}                                                        # Set up three dictionaries to store the predicted y-values for the test names in
  pred_dict_ngram = {}
  pred_dict_lexical = {}

  skf = StratifiedKFold(n_splits = len(df), shuffle=True, random_state=17042020)      # Set up a stratified 5-fold splitter

  for train_index, test_index in skf.split(df, df[['name_type']]):              # For every fold (stratified on the name type, i.e., real, madeup, or talking)
    
    x_train_unfasttexted = df.iloc[train_index]                                 # Split the data into x_train and x_test
    x_test_unfasttexted = df.iloc[test_index]

    x_train_unf_lexical = x_train_unfasttexted[x_train_unfasttexted.name_type != 'madeup']
    x_test_unf_lexical = x_test_unfasttexted[x_test_unfasttexted.name_type != 'madeup']

    y_train = df.iloc[train_index][rating]                                      # Split the data into y_train and y_test
    y_test = df.iloc[test_index][rating]

    y_train_lexical = df.iloc[train_index][df.name_type != 'madeup'][rating] 
    y_test_lexical = df.iloc[test_index][df.name_type != 'madeup'][rating]
    
    test_names = df.iloc[test_index]['name']                                    # Create a list of names in the test set
    test_name_types = df.iloc[test_index]['name_type']                          # Create a list of name types corresponding to the names in the test set

    test_names_lexical = df.iloc[test_index][df.name_type != 'madeup']['name']                                    
    test_name_types_lexical = df.iloc[test_index][df.name_type != 'madeup']['name_type']  

    x_train = fasttext_xifyer(x_train_unfasttexted)                             # Get the combined embeddings (train)
    x_train_ngram = fasttext_xifyer_ngram(x_train_unfasttexted)           # And get the surface_form-less embeddings (train)
    x_train_lexical = fasttext_xifyer_lexical(x_train_unf_lexical)           # And get the surface_form-only embeddings (train)

    x_test = fasttext_xifyer(x_test_unfasttexted)                               # Get the combined embeddings (test)
    x_test_ngram = fasttext_xifyer_ngram(x_test_unfasttexted)             # And get the surface_form-less embeddings (test)
    x_test_lexical = fasttext_xifyer_lexical(x_test_unf_lexical)             # And get the surface_form-only embeddings (train)


    alpha, l1_ratio, n_iters, intercept, mae_test, r2, mae_madeup, mae_real, \
    mae_talking, mean_vec_mae_test, mean_vec_r2 \
    = elasticnetifyer(x_train, y_train, x_test, y_test, 
                      pred_dict_combined, test_names, test_name_types)  # Train the ElasticNetCV given the fold, and return all of the variables of interest (combined)
    
    dictionary['combined'].append([alpha, l1_ratio, n_iters, intercept, mae_test, 
                                   r2, mae_madeup, mae_real, mae_talking, 
                                   mean_vec_mae_test, mean_vec_r2])        # Append the variables of interest to the dictionary
    
    alpha_ngram, l1_ratio_ngram, n_iters_ngram, intercept_ngram, \
    mae_test_ngram, r2_ngram, mae_madeup_ngram, mae_real_ngram, \
    mae_talking_ngram, mean_vec_mae_test_ngram, mean_vec_r2_ngram \
    = elasticnetifyer(x_train_ngram, y_train, x_test_ngram, y_test, 
                      pred_dict_ngram, test_names, test_name_types)          # Train the ElasticNetCV given the fold, and return all of the variables of interest (ngram)

    dictionary['ngram'].append([alpha_ngram, l1_ratio_ngram, n_iters_ngram,
                                intercept_ngram, mae_test_ngram, r2_ngram,
                                mae_madeup_ngram, mae_real_ngram, mae_talking_ngram,
                                mean_vec_mae_test_ngram, mean_vec_r2_ngram])   # Append the variables of interest to the dictionary

    alpha_lexical, l1_ratio_lexical, n_iters_lexical, intercept_lexical, \
    mae_test_lexical, r2_lexical, mae_madeup_lexical, mae_real_lexical, \
    mae_talking_lexical, mean_vec_mae_test_lexical, mean_vec_r2_lexical \
    = elasticnetifyer(x_train_lexical, y_train_lexical, x_test_lexical, y_test_lexical, 
                      pred_dict_lexical, test_names_lexical, test_name_types_lexical, True)    # train the ElasticNetCV given the fold, and return all of the variables of interest (lexical)

    dictionary['lexical'].append([alpha_lexical, l1_ratio_lexical, n_iters_lexical,
                                  intercept_lexical, mae_test_lexical, r2_lexical,
                                  mae_madeup_lexical, mae_real_lexical, mae_talking_lexical,
                                  mean_vec_mae_test_lexical, mean_vec_r2_lexical])   # Append the variables of interest to the dictionary
  
  return pred_dict_combined, pred_dict_ngram, pred_dict_lexical

In [ ]:
def nested_cv_addotron(dictionary, name):
  # Input:
  # - dictionary = the dictionary of values for the dimension of interest
  # - name = string indicating the dimension of interest (i.e., age, gender, or 
  # polarity). This is used to make printing easier and more organised.

  # Process:
  # Given the dimension of interest, for all folds, average the scores and 
  # save all of the variables for both the 'combined', 'ngram', and 'lexical'
  # models as a .csv file

  # Output:
  # Nothing, but all of the calculated metrics for all of the models are stored 
  # in a .csv file.

  column_list = ('dimension', 'analysis_type', 'alpha', 'l1_ratio', 'n_iters',  # List indicating all of the variables of interest
                 'intercept', 'mae_test', 'r2', 'mae_madeup', 'mae_real', 
                 'mae_talking', 'mean_vec_mae_test', 'mean_vec_r2', 'sd_r2', 
                 'sd_mae_total', 'sd_mae_madeup', 'sd_mae_real', 'sd_mae_talking', 
                 'sd_mean_vec_mae', 'sd_mean_vec_r2')
  
  type_list = ['combined', 'ngram', 'lexical']                              # List indicating the model type

  combined_list = dictionary['combined']
  ngram_list = dictionary['ngram']
  lexical_list = dictionary['lexical']

  n_splits = len(dictionary['combined'][0])

  sd_dict_en = {}

  for analysis_type in type_list:    
    sd_r2 = []

    sd_mae_total = []
    sd_mae_madeup = []
    sd_mae_real = []
    sd_mae_talking = []

    sd_mean_vec_mae = []
    sd_mean_vec_r2 = []

    for iteration in dictionary[analysis_type]:
      sd_r2.append(iteration[5])

      sd_mae_total.append(iteration[4])
      sd_mae_madeup.append(iteration[6])
      sd_mae_real.append(iteration[7])
      sd_mae_talking.append(iteration[8])

      sd_mean_vec_mae.append(iteration[9])
      sd_mean_vec_r2.append(iteration[10])

    sd_dict_en[analysis_type]['r2'] = np.sd(sd_r2)
    sd_dict_en[analysis_type]['total'] = np.sd(sd_mae_total)
    sd_dict_en[analysis_type]['madeup'] = np.sd(sd_mae_madeup)
    sd_dict_en[analysis_type]['real'] = np.sd(sd_mae_real)
    sd_dict_en[analysis_type]['talking'] = np.sd(sd_mae_talking)
    sd_dict_en[analysis_type]['mean_vec_mae'] = np.sd(sd_mean_vec_mae)
    sd_dict_en[analysis_type]['mean_vec_r2'] = np.sd(sd_mean_vec_r2)

  
  combined_list = [sum(x) for x in zip(*combined_list)]                           # For all of the variables in the list, sum the scores (combined)
  combined_list = [x / n_splits for x in combined_list]                                  # For all of the variables in the list, divide the sum by 5 to get the average score (combined)

  ngram_list = [sum(x) for x in zip(*ngram_list)]                         # For all of the variables in the list, sum the scores (ngram)
  ngram_list = [x / n_splits for x in ngram_list]                                # For all of the variables in the list, divide the sum by 5 to get the average score (ngram)

  lexical_list = [sum(filter(None,x))  for x in zip(*lexical_list)]
  lexical_list = [x / n_splits for x in lexical_list]

  csv_df = []                                                                   # Create a list of lists that will be converted to a data frame

  for value_list, analysis_type in zip([combined_list, ngram_list,              # Given the list of metrics for every model (combined, ngram, lexical)
                                        lexical_list], type_list):
    value_list.insert(0, analysis_type)                                         # Insert the name of the analysis type to the values (i.e., 'combined', etc.)
    value_list.insert(0, name)                                                  # Insert the name of the dimension (i.e., 'age', 'gender', 'polarity') to the values
    value_list.append(sd_dict_en[analysis_type]['r2'])
    value_list.append(sd_dict_en[analysis_type]['total'])
    value_list.append(sd_dict_en[analysis_type]['madeup'])
    value_list.append(sd_dict_en[analysis_type]['real'])
    value_list.append(sd_dict_en[analysis_type]['talking'])
    value_list.append(sd_dict_en[analysis_type]['mean_vec_mae'])
    value_list.append(sd_dict_en[analysis_type]['mean_vec_r2'])
    csv_df.append(value_list)                                                   # Add the list of values as a row to the data frame
  
  csv_df = pd.DataFrame(csv_df, columns=column_list)                            # Convert list of lists to dataframe, add column names
  csv_df.to_csv(csv_path + name +'_en_metrics.csv', index=False)                # Save DF as .csv file

##### Running the functions

In [ ]:
age_dict = {'combined' : [], 'ngram' : [], 'lexical' : []}                  # initialize the score dictionary for age

age_pred_dict_combined, age_pred_dict_ngram, age_pred_dict_lexical = \
nested_cross_validator(df_age, 'rating.mean_age', age_dict)                     # Perform the 5-fold cross validation

pickle_age_ncv = open(pickle_path + "age_ncv.pickle", "wb")                     # Save the dictionary to a pickle
pickle.dump(age_dict, pickle_age_ncv)
pickle_age_ncv.close()

nested_cv_addotron(age_dict, "age")                                             # Save all of the metrics as a .csv file

In [ ]:
age_pred_en_df = []                                                             # Create a list of lists that will be converted to a dataframe

for i in sorted(df_age['name']):                                                # For every name in the dataframe
  i = i.lower()
  
  try:
    age_pred_en_df.append([i, age_pred_dict_combined[i][0],                        # Append the name, name type, and normal + mean_vector only predictions for the 
                          age_pred_dict_combined[i][1],                           # three model types as a row to the list of lists
                          age_pred_dict_combined[i][2],
                          age_pred_dict_combined[i + '_mean_vector'][2], 
                          age_pred_dict_ngram[i][2], 
                          age_pred_dict_ngram[i + '_mean_vector'][2], 
                          age_pred_dict_lexical[i][2],
                          age_pred_dict_lexical[i + '_mean_vector'][2]])        
    
  except KeyError:
    age_pred_en_df.append([i, age_pred_dict_combined[i][0],                        # Append the name, name type, and normal + mean_vector only predictions for the 
                         age_pred_dict_combined[i][1],                           # three model types as a row to the list of lists
                         age_pred_dict_combined[i][2],
                         age_pred_dict_combined[i + '_mean_vector'][2], 
                         age_pred_dict_ngram[i][2], 
                         age_pred_dict_ngram[i + '_mean_vector'][2], 
                         None, None]) 

age_pred_en_df = pd.DataFrame(age_pred_en_df, columns=column_list_pred)         # Convert list of lists to DF

age_pred_en_df.to_csv(csv_path + 'age_pred_en.csv', index=False)             # Save DF as .csv

In [ ]:
gender_dict = {'combined' : [], 'ngram' : [], 'lexical' : []}               # initialize the score dictionary for gender

gender_pred_dict_combined, gender_pred_dict_ngram, \
gender_pred_dict_lexical = \
nested_cross_validator(df_gender, 'rating.mean_gender', gender_dict)            # Perform the 5-fold cross validation

pickle_gender_ncv = open(pickle_path + "gender_ncv.pickle", "wb")               # Save the dictionary to a pickle
pickle.dump(gender_dict, pickle_gender_ncv)
pickle_gender_ncv.close()

nested_cv_addotron(gender_dict, "gender")                                       # Save all of the metrics as a .csv file

In [ ]:
gender_pred_en_df = []                                                          # Create a list of lists that will be converted to a dataframe

for i in sorted(df_gender['name']):                                             # For every name in the dataframe
  i = i.lower()
  
  try:
    gender_pred_en_df.append([i, gender_pred_dict_combined[i][0],                  # Append the name, name type, and normal + mean_vector only predictions for the
                              gender_pred_dict_combined[i][1],                     # three model types as a row to the list of lists
                              gender_pred_dict_combined[i][2],
                              gender_pred_dict_combined[i + '_mean_vector'][2], 
                              gender_pred_dict_ngram[i][2], 
                              gender_pred_dict_ngram[i + '_mean_vector'][2], 
                              gender_pred_dict_lexical[i][2],
                              gender_pred_dict_lexical[i + '_mean_vector'][2]])
    
  except KeyError:
    gender_pred_en_df.append([i, gender_pred_dict_combined[i][0],                  # Append the name, name type, and normal + mean_vector only predictions for the
                            gender_pred_dict_combined[i][1],                     # three model types as a row to the list of lists
                            gender_pred_dict_combined[i][2],
                            gender_pred_dict_combined[i + '_mean_vector'][2], 
                            gender_pred_dict_ngram[i][2], 
                            gender_pred_dict_ngram[i + '_mean_vector'][2], 
                            None, None])
  
gender_pred_en_df = pd.DataFrame(gender_pred_en_df, columns=column_list_pred)   # Convert list of lists to DF

gender_pred_en_df.to_csv(csv_path + 'gender_pred_en.csv', index=False)       # Save DF as .csv

In [ ]:
polarity_dict = {'combined' : [], 'ngram' : [], 'lexical' : []}             # initialize the score dictionary for polarity

polarity_pred_dict_combined, polarity_pred_dict_ngram, \
polarity_pred_dict_lexical = \
nested_cross_validator(df_polarity, 'rating.mean_valence', polarity_dict)       # Perform the 5-fold cross validation, save the prediction dictionaries

pickle_polarity_ncv = open(pickle_path + "polarity_ncv.pickle", "wb")           # Save the dictionary to a pickle
pickle.dump(polarity_dict, pickle_polarity_ncv)
pickle_polarity_ncv.close()

nested_cv_addotron(polarity_dict, "polarity")                                   # Save all of the metrics as a .csv file

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
polarity_pred_en_df = []                                                        # Create a list of lists that will be converted to a dataframe

for i in sorted(df_polarity['name']):                                           # For every name in the dataframe        
  i = i.lower()

  try:
    polarity_pred_en_df.append([i, polarity_pred_dict_combined[i][0],              # Append the name, name type, and normal + mean_vector only predictions for the
                                polarity_pred_dict_combined[i][1],                 # three model types as a row to the list of lists
                                polarity_pred_dict_combined[i][2],
                                polarity_pred_dict_combined[i + '_mean_vector'][2], 
                                polarity_pred_dict_ngram[i][2], 
                                polarity_pred_dict_ngram[i + '_mean_vector'][2], 
                                polarity_pred_dict_lexical[i][2],
                                polarity_pred_dict_lexical[i + '_mean_vector'][2]])
    
  except KeyError:
    polarity_pred_en_df.append([i, polarity_pred_dict_combined[i][0],              # Append the name, name type, and normal + mean_vector only predictions for the
                              polarity_pred_dict_combined[i][1],                 # three model types as a row to the list of lists
                              polarity_pred_dict_combined[i][2],
                              polarity_pred_dict_combined[i + '_mean_vector'][2], 
                              polarity_pred_dict_ngram[i][2], 
                              polarity_pred_dict_ngram[i + '_mean_vector'][2], 
                              None, None])
  
polarity_pred_en_df = pd.DataFrame(polarity_pred_en_df, columns=column_list_pred) # Convert list of lists to DF

polarity_pred_en_df.to_csv(csv_path + 'polarity_pred_en.csv', index=False)   # Save DF as .csv

### Neural Networks

In [ ]:
def fnn_maker(x_train, y_train, x_test, y_test, nodes, dropout):
  # Input:
  # - x_train = array of embeddings used to train the model
  # - y_train = array of ratings used to train the model
  # - x_test = array of embeddings used to test the model
  # - y_test = array of ratings used to test the model
  # - nodes = integer indicating the number of nodes to use in the hidden layer
  # - dropout = integer indicating the amount of dropout to use in the hidden layer

  # Process:
  # Train a sequential NN using the train set and return the model & test MSE

  # Output:
  # - mse = test set mean squared error
  # - fnn_model = trained neural network model

  random.seed(17042020)                                                         # Set the seed using python's built-in seed function
  set_seed(17042020)                                                            # Set the seed using keras/tensorflow's seed function, just to be sure

  fnn_model = Sequential()                                                      # Initialize a sequential NN

  fnn_model.add(Dense(nodes, input_dim=300, kernel_initializer=HeNormal(), 
                      activation=keras.layers.LeakyReLU()))                     # Add a dense layer with the specified nodes
  fnn_model.add(Dropout(dropout))

  fnn_model.add(Dense(1, activation='linear'))                                  # Add a final layer

  callback = EarlyStopping(monitor = 'loss', patience=3)                        # Add early stopping that stops after 3 rounds without improvement

  fnn_model.compile(optimizer=Adam(), loss='mean_squared_error')                # Compile the model with mean squared loss

  fnn_model.fit(x_train, y_train, epochs=100, batch_size=len(x_train), 
                callbacks=[callback], verbose=0)                                # Fit the model on the train set

  y_pred = fnn_model.predict(x_test)                                            # Feed the test cases to the model to retrieve predictions
  
  mse = mean_squared_error(y_test, y_pred)                                      # Calculate test-set MSE

  return mse, fnn_model

  #return fnn_model

#### Grid Search

In [ ]:
def nested_neural_network_finder(df, rating, dimension, dictionary):
  # Input:
  # - df = a dataframe with the name, name_type, and rating for the dimension at  
  # hand (i.e., age, gender, or polarity)
  # - rating = a string indicating what rating to extract from the df
  # - dimension = a string indicating what dimension is considered (i.e., 'age',
  # 'gender', or 'polarity')
  # - dictionary = an empty dictionary to store the MSE output by fnn_maker() in 
  # per configuration

  # Process:
  # Given the df, get 5 train/test splits, and per fold, train a LOOCV NN model
  # using fnn_maker() for the combined and ngram data. Then, print the scores.

  # Output: 
  # Nothing, but the MSE for all different configurations in the grid search are
  # printed

  #### NOTE: For the 'extra' NN grid search, change the list of nodes and 
  #### dropout values to search and print!! I am not making this an extra 
  #### function because I already have enough and it's simple to understand.
  q=1

  skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=17042020)      # Set up a stratified 5-fold splitter

  for train_index, test_index in skf.split(df, df[['name_type']]):              # For every fold (stratified on the name type)
    print("Fold:", q, "Time:", time.strftime("%D:%H:%M:%S", time.localtime()))
    k_fold = KFold(n_splits = len(train_index), shuffle=True, random_state=17042020) # Set up a LOOCV splitter

    x_train_unfasttexted = df.iloc[train_index]                                 # Split the data into x_train and x_test
    x_test_unfasttexted = df.iloc[test_index]

    x_train = fasttext_xifyer(x_train_unfasttexted)                             # Get the word embeddings
    x_train_ngram = fasttext_xifyer_ngram(x_train_unfasttexted)
    x_train_lexical = fasttext_xifyer_lexical(x_train_unfasttexted)

    x_test = fasttext_xifyer(x_test_unfasttexted)
    x_test_ngram = fasttext_xifyer_ngram(x_test_unfasttexted)
    x_test_ngram = fasttext_xifyer_lexical(x_test_unfasttexted)

    y_train = df.iloc[train_index][rating]                                      # Split the data into y_train and y_test
    y_test = df.iloc[test_index][rating]

    for nested_train_index, nested_test_index in k_fold.split(x_train):         # For every fold in the LOOCV splitter
      nested_x_train = x_train[[nested_train_index], :].reshape(len(nested_train_index), 300)   # Get a nested x_train and x_test
      nested_x_train_ngram = x_train_ngram[[nested_train_index], :].reshape(len(nested_train_index), 300)
      nested_x_train_lexical = x_train_lexical[[nested_train_index], :].reshape(len(nested_train_index), 300)

      nested_x_test = x_train[[nested_test_index], :].reshape(len(nested_test_index), 300)
      nested_x_test_ngram = x_train_ngram[[nested_test_index], :].reshape(len(nested_test_index), 300)
      nested_x_test_lexical = x_train_lexical[[nested_test_index], :].reshape(len(nested_test_index), 300)

      nested_y_train = y_train.iloc[nested_train_index]                         # Get a nested y_train and y_test
      nested_y_test = y_train.iloc[nested_test_index]

      for nodes in [128, 256, 300, 512]:                                        # For every number of nodes considered
        for dropout in [0.2, 0.3, 0.4, 0.5, 0.6]:                               # For every initialization of dropout considered
          dictionary['combined'][str(nodes)][str(dropout)].append(
              fnn_maker(nested_x_train, 
                        nested_y_train, 
                        nested_x_test, 
                        nested_y_test, 
                        nodes, 
                        dropout))                                               # Train a NN using the combined data and append the scores to a dict

          dictionary['ngram'][str(nodes)][str(dropout)].append(
              fnn_maker(nested_x_train_ngram, 
                        nested_y_train, 
                        nested_x_test_ngram, 
                        nested_y_test, 
                        nodes, 
                        dropout))                                               # Train a NN using the ngram data and append the scores to a dict
                  
          dictionary['lexical'][str(nodes)][str(dropout)].append(
              fnn_maker(nested_x_train_lexical, 
                        nested_y_train, 
                        nested_x_test_lexical, 
                        nested_y_test, 
                        nodes, 
                        dropout))                                               # Train a NN using the ngram data and append the scores to a dict
    q += 1
  
  for nodes in [128, 256, 300, 512]:                                            # Given all configurations, print the combined and ngram MSE
      for dropout in [0.2, 0.3, 0.4, 0.5, 0.6]:
        combined_mse = sum(dictionary['combined'][str(nodes)][str(dropout)]) / len(dictionary['combined'][str(nodes)][str(dropout)])
        ngram_mse = sum(dictionary['ngram'][str(nodes)][str(dropout)]) / len(dictionary['ngram'][str(nodes)][str(dropout)])
        lexical_mse = sum(dictionary['lexical'][str(nodes)][str(dropout)]) / len(dictionary['lexical'][str(nodes)][str(dropout)])

        print("{}, combined, nodes = {}, dropout = {}, Average MSE = {}".format(dimension, nodes, dropout, combined_mse))
        print("{}, ngram, nodes = {}, dropout = {}, Average MSE = {}".format(dimension, nodes, dropout, ngram_mse))
        print("{}, lexical, nodes = {}, dropout = {}, Average MSE = {}".format(dimension, nodes, dropout, lexical_mse))
        print("\n")

In [ ]:
age_dict_nn = {'combined' : {'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}}, 
               'ngram' :{'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}},
               'lexical' :{'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}}} # Initialize a dictionary to store values in

nested_neural_network_finder(df_age, 'rating.mean_age', 'Age', age_dict_nn)     # Run the nested neural network finder function

Fold: 1 Time: 04/27/22:15:25:25


KeyboardInterrupt: ignored

In [ ]:
gender_dict_nn =  {'combined' : {'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}}, 
               'ngram' :{'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}},
               'lexical' :{'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}}}  # Initialize a dictionary to store values in

nested_neural_network_finder(df_gender, 'rating.mean_gender', 'Gender', gender_dict_nn) # Run the nested neural network finder function

Fold: 1 Time: 04/27/22:15:26:00
Fold: 2 Time: 04/27/22:19:17:26


In [ ]:
polarity_dict_nn =  {'combined' : {'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}}, 
               'ngram' :{'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}},
               'lexical' :{'128': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '256': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '300': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}, 
                            '512': {'0.2': [], '0.3': [], '0.4': [], '0.5': [], '0.6': []}}} # Initialize a dictionary to store values in

nested_neural_network_finder(df_polarity, 'rating.mean_valence', 'Polarity', polarity_dict_nn) # Run the nested neural network finder function

Fold: 1 Time: 04/27/22:06:58:16
Fold: 2 Time: 04/27/22:07:31:31
Fold: 3 Time: 04/27/22:08:05:26
Fold: 4 Time: 04/27/22:08:37:19
Fold: 5 Time: 04/27/22:09:08:37
Polarity, regular, nodes = 128, dropout = 0.2, Average MSE = 321.10995032291936
Polarity, formless, nodes = 128, dropout = 0.2, Average MSE = 328.2864228138766
Polarity, formonly, nodes = 128, dropout = 0.2, Average MSE = 353.7593838626403


Polarity, regular, nodes = 128, dropout = 0.3, Average MSE = 339.67209947406394
Polarity, formless, nodes = 128, dropout = 0.3, Average MSE = 340.7091010832673
Polarity, formonly, nodes = 128, dropout = 0.3, Average MSE = 354.6769563212269


Polarity, regular, nodes = 128, dropout = 0.4, Average MSE = 347.98369395705856
Polarity, formless, nodes = 128, dropout = 0.4, Average MSE = 348.79425140161845
Polarity, formonly, nodes = 128, dropout = 0.4, Average MSE = 353.8075563756581


Polarity, regular, nodes = 128, dropout = 0.5, Average MSE = 360.6970957046829
Polarity, formless, nodes = 128, d

#### Evaluation of Final Model

##### Functions

In [ ]:
def fnn_maker_and_evaluator(x_train, y_train, x_test, y_test, pred_dict, 
                            test_names, test_name_types, nodes, dropout, lexical = None):
  # Input:
  # - x_train = array of embeddings used to train the model
  # - y_train = array of ratings used to train the model
  # - x_test = array of embeddings used to test the model
  # - y_test = array of ratings used to test the model
  # - pred_dict = a dictionary that will be filled with predictions per name
  # - test_names = dataframe containing the full names (i.e., not the embeddings) 
  # - test_name_types = dataframe containing the name type (real, madeup, talking)
  # - nodes = integer indicating the number of nodes to use in the hidden layer
  # - dropout = integer indicating the amount of dropout to use in the hidden layer

  # Process:
  # Train a sequential NN using the train set using fnn_maker(). Then, evaluate
  # the model using model_evaluator and return the values.

  # Output:
  # Too many to explain here. Basically, a bunch of metrics to test the model.

  _, fnn_model = fnn_maker(x_train, y_train, x_test, y_test, nodes, dropout)
  
  mae_test,r2, mae_madeup, mae_real, mae_talking, mean_vec_mae_test, mean_vec_r2 \
  = model_evaluator(fnn_model, x_train, x_test, y_train, y_test, 
                    pred_dict, test_names, test_name_types, lexical)

  return mae_test, r2, mae_madeup, mae_real, mae_talking, mean_vec_mae_test, mean_vec_r2 

In [ ]:
def neural_network_5_folder(df, rating, dimension, dictionary, nodes, dropout):
  # Input:
  # - df = a dataframe with the name, name_type, and rating for the dimension at  
  # hand (i.e., age, gender, or polarity)
  # - rating = a string indicating what rating to extract from the df
  # - dimension = a string indicating what dimension is considered (i.e., 'age',
  # 'gender', or 'polarity')
  # - dictionary = an empty dictionary to store the MSE output by fnn_maker() in 
  # per configuration
  # - nodes = integer indicating the number of nodes to use in the hidden layer
  # - dropout = integer indicating the amount of dropout to use in the hidden layer

  # Process:
  # Given the df, get 5 train/test splits, and per fold, train a NN model using
  # fnn_maker() for the combined, ngram, and lexical data. Then, store these
  # metrics in a .csv file.

  # Output: 
  # - pred_dict_combined: y_true and y_pred (for several conditions) per name (combined)
  # - pred_dict_ngram: y_true and y_pred (for several conditions) per name (ngram)
  # - pred_dict_lexical: y_true and y_pred (for several conditions) per name (lexical)

  skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=17042020)      # Set up a stratified 5-fold splitter
  
  pred_dict_combined = {}                                                        # Set up three dictionaries to store the predicted y-values for the test names in
  pred_dict_ngram = {}
  pred_dict_lexical = {}

  for train_index, test_index in skf.split(df, df[['name_type']]):              # For every fold (stratified on the name type, i.e., real, madeup, or talking)
    x_train_unfasttexted = df.iloc[train_index]                                 # Split the data into x_train and x_test
    x_test_unfasttexted = df.iloc[test_index]

    x_train_unf_lexical = x_train_unfasttexted[x_train_unfasttexted.name_type != 'madeup']
    x_test_unf_lexical = x_test_unfasttexted[x_test_unfasttexted.name_type != 'madeup']

    x_train = fasttext_xifyer(x_train_unfasttexted)                             # Get the word embeddings
    x_train_ngram = fasttext_xifyer_ngram(x_train_unfasttexted)
    x_train_lexical = fasttext_xifyer_lexical(x_train_unf_lexical)

    x_test = fasttext_xifyer(x_test_unfasttexted)
    x_test_ngram = fasttext_xifyer_ngram(x_test_unfasttexted)
    x_test_lexical = fasttext_xifyer_lexical(x_test_unf_lexical)


    y_train = df.iloc[train_index][rating]                                      # Split the data into y_train and y_test
    y_test = df.iloc[test_index][rating]

    y_train_lexical = df.iloc[train_index][df.name_type != 'madeup'][rating] 
    y_test_lexical = df.iloc[test_index][df.name_type != 'madeup'][rating]


    test_names = df.iloc[test_index]['name']                                    # Get a list of the names in the test set
    test_name_types = df.iloc[test_index]['name_type']                          # Get a list of name types corresponding to the names in the test set

    test_names_lexical = df.iloc[test_index][df.name_type != 'madeup']['name']                                    
    test_name_types_lexical = df.iloc[test_index][df.name_type != 'madeup']['name_type']                          


    mae_test, r2, mae_madeup, mae_real, mae_talking, mean_vec_mae_test, mean_vec_r2 \
    = fnn_maker_and_evaluator(x_train, 
                              y_train, 
                              x_test, 
                              y_test, 
                              pred_dict_combined,
                              test_names,
                              test_name_types, 
                              nodes,
                              dropout)                                          # Train the NN given the fold, and return all of the variables of interest (combined)

    mae_test_ngram, r2_ngram, mae_madeup_ngram, mae_real_ngram, mae_talking_ngram, \
    mean_vec_mae_test_ngram, mean_vec_r2_ngram \
    = fnn_maker_and_evaluator(x_train_ngram, 
                              y_train, 
                              x_test_ngram, 
                              y_test, 
                              pred_dict_ngram,
                              test_names,
                              test_name_types, 
                              nodes,
                              dropout)                                          # Train the NN given the fold, and return all of the variables of interest (ngram)

    mae_test_lexical, r2_lexical, mae_madeup_lexical, mae_real_lexical, mae_talking_lexical, \
    mean_vec_mae_test_lexical, mean_vec_r2_lexical \
    = fnn_maker_and_evaluator(x_train_lexical, 
                              y_train_lexical, 
                              x_test_lexical, 
                              y_test_lexical, 
                              pred_dict_lexical,
                              test_names_lexical,
                              test_name_types_lexical, 
                              nodes,
                              dropout, 
                              lexical = True)                                       # Train the NN given the fold, and return all of the variables of interest (lexical)

    dictionary['combined'].append([mae_test, r2, mae_madeup, mae_real, mae_talking, 
                                  mean_vec_mae_test, mean_vec_r2])                    # Append the variables of interest to the dictionary

    dictionary['ngram'].append([mae_test_ngram, r2_ngram, mae_madeup_ngram, 
                                mae_real_ngram, mae_talking_ngram, 
                                mean_vec_mae_test_ngram, mean_vec_r2_ngram])    # Append the variables of interest to the dictionary
    
    dictionary['lexical'].append([mae_test_lexical, r2_lexical, mae_madeup_lexical, 
                                  mae_real_lexical, mae_talking_lexical,
                                  mean_vec_mae_test_lexical, mean_vec_r2_lexical])  # Append the variables of interest to the dictionary
  
  column_list = ('dimension', 'analysis_type', 'mae_test', 'r2', 'mae_madeup', 
                 'mae_real', 'mae_talking', 'mean_vec_mae_test', 'mean_vec_r2',
                 'sd_r2', 'sd_mae_total', 'sd_mae_madeup', 'sd_mae_real', 
                 'sd_mae_talking', 'sd_mean_vec_mae', 'sd_mean_vec_r2')         # List indicating all of the variables of interest 
  
  type_list = ['combined', 'ngram', 'lexical']                                  # List indicating the model type

  sd_dict_nn = {}

  for analysis_type in type_list:    
    sd_r2 = []

    sd_mae_total = []
    sd_mae_madeup = []
    sd_mae_real = []
    sd_mae_talking = []

    sd_mean_vec_mae = []
    sd_mean_vec_r2 = []

    for iteration in dictionary[analysis_type]:
      sd_r2.append(iteration[1])

      sd_mae_total.append(iteration[0])
      sd_mae_madeup.append(iteration[2])
      sd_mae_real.append(iteration[3])
      sd_mae_talking.append(iteration[4])

      sd_mean_vec_mae.append(iteration[5])
      sd_mean_vec_r2.append(iteration[6])

    sd_dict_nn[analysis_type]['r2'] = np.sd(sd_r2)
    sd_dict_nn[analysis_type]['total'] = np.sd(sd_mae_total)
    sd_dict_nn[analysis_type]['madeup'] = np.sd(sd_mae_madeup)
    sd_dict_nn[analysis_type]['real'] = np.sd(sd_mae_real)
    sd_dict_nn[analysis_type]['talking'] = np.sd(sd_mae_talking)
    sd_dict_nn[analysis_type]['mean_vec_mae'] = np.sd(sd_mean_vec_mae)
    sd_dict_nn[analysis_type]['mean_vec_r2'] = np.sd(sd_mean_vec_r2)

  combined_list = dictionary['combined']
  ngram_list = dictionary['ngram']
  lexical_list = dictionary['lexical']
  
  combined_list = [sum(x) for x in zip(*combined_list)]                           # For all of the variables in the list, sum the scores 
  combined_list = [x / 5 for x in combined_list]                                  # For all of the variables in the list, divide the sum by 5 to get the average score 

  ngram_list = [sum(x) for x in zip(*ngram_list)]                         
  ngram_list = [x / 5 for x in ngram_list]                                

  lexical_list = [sum(filter(None,x)) for x in zip(*lexical_list)]                         
  lexical_list = [x / 5 for x in lexical_list]

  csv_df = []                                                                   # Create a list of lists that will be converted to a dataframe

  for value_list, analysis_type in zip([combined_list, ngram_list,            # Given the list of metrics for every model (combined, ngram, lexical)
                                        lexical_list], type_list):
    value_list.insert(0, analysis_type)                                         # Insert the name of the analysis type to the values (i.e., 'combined', etc.)
    value_list.insert(0, dimension)                                             # Insert the name of the dimension (i.e., 'age', 'gender', 'polarity') to the values
    value_list.append(sd_dict_nn[analysis_type]['r2'])
    value_list.append(sd_dict_nn[analysis_type]['total'])
    value_list.append(sd_dict_nn[analysis_type]['madeup'])
    value_list.append(sd_dict_nn[analysis_type]['real'])
    value_list.append(sd_dict_nn[analysis_type]['talking'])
    value_list.append(sd_dict_nn[analysis_type]['mean_vec_mae'])
    value_list.append(sd_dict_nn[analysis_type]['mean_vec_r2'])
    csv_df.append(value_list)                                                   # Add the list of values as a row to the DF list of lists
  
  csv_df = pd.DataFrame(csv_df, columns=column_list)                            # Convert the list of lists to a DF
  csv_df.to_csv(csv_path + dimension +'_nn_metrics.csv', index=False)           # Save the DF as a .csv file

  return pred_dict_combined, pred_dict_ngram, pred_dict_lexical

##### Running the functions

In [ ]:
age_dict_nn_final = {'combined' : [], 'ngram' : [], 'lexical': []}          # initialize the score dictionary for age

age_pred_dict_combined_nn, age_pred_dict_ngram_nn, \
age_pred_dict_lexical_nn = \
neural_network_5_folder(df_age, 'rating.mean_age', 'age', 
                        age_dict_nn_final, 300, 0.5)                            # Perform the 5-fold cross validation and save the metrics as a .csv file

pickle_age_nn_final = open(pickle_path + "age_nn_final.pickle", "wb")           # Save the dictionary to a pickle
pickle.dump(age_dict_nn_final, pickle_age_nn_final)
pickle_age_nn_final.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
age_pred_nn_df = []                                                             # Create a list of lists that will be converted to a dataframe

for i in sorted(df_age['name']):                                                # For every name in the dataframe
  i = i.lower()

  try:
    age_pred_nn_df.append([i, age_pred_dict_combined_nn[i][0],                     # Append the name, name type, and normal + mean_vector only predictions for the
                          age_pred_dict_combined_nn[i][1],                        # three model types as a row to the list of lists
                          age_pred_dict_combined_nn[i][2],
                          age_pred_dict_combined_nn[i + '_mean_vector'][2], 
                          age_pred_dict_ngram_nn[i][2], 
                          age_pred_dict_ngram_nn[i + '_mean_vector'][2], 
                          age_pred_dict_lexical_nn[i][2],
                          age_pred_dict_lexical_nn[i + '_mean_vector'][2]])
    
  except KeyError:
    age_pred_nn_df.append([i, age_pred_dict_combined_nn[i][0],                     # Append the name, name type, and normal + mean_vector only predictions for the
                          age_pred_dict_combined_nn[i][1],                        # three model types as a row to the list of lists
                          age_pred_dict_combined_nn[i][2],
                          age_pred_dict_combined_nn[i + '_mean_vector'][2], 
                          age_pred_dict_ngram_nn[i][2], 
                          age_pred_dict_ngram_nn[i + '_mean_vector'][2], 
                          None, None])
  
age_pred_nn_df = pd.DataFrame(age_pred_nn_df, columns=column_list_pred)         # Convert list of lists to DF

age_pred_nn_df.to_csv(csv_path + 'age_pred_nn.csv', index=False)                # Save DF as .csv

In [ ]:
gender_dict_nn_final = {'combined' : [], 'ngram' : [], 'lexical': []}       # initialize the score dictionary for gender

gender_pred_dict_combined_nn, gender_pred_dict_ngram_nn, \
gender_pred_dict_lexical_nn = \
neural_network_5_folder(df_gender, 'rating.mean_gender', 'gender', 
                        gender_dict_nn_final, 512, 0.3)                         # Perform the 5-fold cross validation and save the metrics as a .csv file

pickle_gender_nn_final = open(pickle_path + "gender_nn_final.pickle", "wb")     # Save the dictionary to a pickle
pickle.dump(gender_dict_nn_final, pickle_gender_nn_final)
pickle_gender_nn_final.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
gender_pred_nn_df = []                                                          # Create a list of lists that will be converted to a dataframe

for i in sorted(df_gender['name']):                                             # For every name in the dataframe
  i = i.lower()
  try:
    gender_pred_nn_df.append([i, gender_pred_dict_combined_nn[i][0],               # Append the name, name type, and normal + mean_vector only predictions for the
                          gender_pred_dict_combined_nn[i][1],                     # three model types as a row to the list of lists
                          gender_pred_dict_combined_nn[i][2],
                          gender_pred_dict_combined_nn[i + '_mean_vector'][2], 
                          gender_pred_dict_ngram_nn[i][2], 
                          gender_pred_dict_ngram_nn[i + '_mean_vector'][2], 
                          gender_pred_dict_lexical_nn[i][2],
                          gender_pred_dict_lexical_nn[i + '_mean_vector'][2]])

  except KeyError:
   gender_pred_nn_df.append([i, gender_pred_dict_combined_nn[i][0],               # Append the name, name type, and normal + mean_vector only predictions for the
                          gender_pred_dict_combined_nn[i][1],                     # three model types as a row to the list of lists
                          gender_pred_dict_combined_nn[i][2],
                          gender_pred_dict_combined_nn[i + '_mean_vector'][2], 
                          gender_pred_dict_ngram_nn[i][2], 
                          gender_pred_dict_ngram_nn[i + '_mean_vector'][2], 
                          None,
                          None])

gender_pred_nn_df = pd.DataFrame(gender_pred_nn_df, columns=column_list_pred)   # Convert list of lists to DF

gender_pred_nn_df.to_csv(csv_path + 'gender_pred_nn.csv', index=False)          # Save DF as .csv 

In [ ]:
polarity_dict_nn_final = {'combined' : [], 'ngram' : [], 'lexical': []}     # initialize the score dictionary for polarity

polarity_pred_dict_combined_nn, polarity_pred_dict_ngram_nn, \
polarity_pred_dict_lexical_nn = \
neural_network_5_folder(df_polarity, 'rating.mean_valence', 'polarity', 
                        polarity_dict_nn_final, 512, 0.5)                       # Perform the 5-fold cross validation and save the metrics as a .csv file

pickle_polarity_nn_final = open(pickle_path + "polarity_nn_final.pickle", "wb") # Save the dictionary to a pickle
pickle.dump(polarity_dict_nn_final, pickle_polarity_nn_final)
pickle_polarity_nn_final.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
polarity_pred_nn_df = []                                                        # Create a list of lists that will be converted to a dataframe

for i in sorted(df_polarity['name']):                                            # For every name in the dataframe
  i = i.lower()
  
  try:
    polarity_pred_nn_df.append([i, polarity_pred_dict_combined_nn[i][0],           # Append the name, name type, and normal + mean_vector only predictions for the
                                polarity_pred_dict_combined_nn[i][1],              # three model types as a row to the list of lists
                                polarity_pred_dict_combined_nn[i][2],
                                polarity_pred_dict_combined_nn[i + '_mean_vector'][2], 
                                polarity_pred_dict_ngram_nn[i][2], 
                                polarity_pred_dict_ngram_nn[i + '_mean_vector'][2], 
                                polarity_pred_dict_lexical_nn[i][2],
                                polarity_pred_dict_lexical_nn[i + '_mean_vector'][2]])
    
  except KeyError:
    polarity_pred_nn_df.append([i, polarity_pred_dict_combined_nn[i][0],           # Append the name, name type, and normal + mean_vector only predictions for the
                                polarity_pred_dict_combined_nn[i][1],              # three model types as a row to the list of lists
                                polarity_pred_dict_combined_nn[i][2],
                                polarity_pred_dict_combined_nn[i + '_mean_vector'][2], 
                                polarity_pred_dict_ngram_nn[i][2], 
                                polarity_pred_dict_ngram_nn[i + '_mean_vector'][2], 
                                None, None])
  
polarity_pred_nn_df = pd.DataFrame(polarity_pred_nn_df, columns=column_list_pred) # Convert list of lists to DF

polarity_pred_nn_df.to_csv(csv_path + 'polarity_pred_nn.csv', index=False)   # Save DF as .csv

#### Adding all metrics CSVs together

In [ ]:
csv_complete = []                                                               # Create list of lists, that will be converted to a DF

for file_name in sorted(os.listdir(csv_path)):                                  # Locate all files in the folder
  if file_name == 'complete_metrics.csv':
    pass
  else:
    if file_name[-11:] == 'metrics.csv':                                          # If the files ends with 'metrics.csv'
      df_temp = pd.read_csv(csv_path + file_name)                                 # Convert the .csv file to a DF

      if file_name[-14:] == 'nn_metrics.csv':                                     # If the file has neural network metrics
        for row in df_temp.values.tolist():                                       # For every row
          row = row[:2] + ['Neural Network'] + row[2:] + [None, None, None, None] # Add none to the metrics that don't apply to neural networks (e.g., alpha, l1_ratio)
          csv_complete.append(row)                                                # Append the row to the list of lists
      else:                                                                       # Else
        columns = list(df_temp.columns)                                           # Save the column names as a list
        columns = columns[:2] + ['model_type'] + columns[6:] + columns[2:6]               
        for row in df_temp.values.tolist():                                       # For every row in the ElasticNet metrics
          row = row[:2] + ['ElasticNet'] + row[6:] + row[2:6]
          csv_complete.append(row)                                                # Append the row to the list of lists

csv_complete = pd.DataFrame(csv_complete, columns=columns)                  # Convert list of lists to a DF
csv_complete.to_csv(csv_path + 'complete_metrics.csv', index=False)             # Save DF as a .csv file

## Plots Playing Ground

Ideas (I'm not listing possible tables for the results, cause those seem like no-brainers to me):

- Dimension Y-axis vs Dimension X-axis, real vs predicted, ANN vs ElasticNet
- Histogram of predicted 'dimensionness' per dimension & regression type
- Histogram of MAE per name type, per dimension & regression type
- Real vs predicted regression per dimension & name type (one of these can be done with different colours/lines, while the other can be done with left/right)
- Distribution of dimensionness for the predicted (per regression type) and the real --> see if they follow similar distributions!
- Schematic depiction of the experimental design (as done by Giovanni in one of his papers)

- **ONLY WHEN ENOUGH TIME**: Permutation testing to find the most influential n-grams per dimension & regression type

Problems:
- 170 names, how do I choose 'exemplary ones'? 
- How to combine dimenions / regression types / real vs predicted, while keeping things interpretable 
  - Scola: old/male = left, young/female = right, dimensions = colour
- 


# Having some fun

In [ ]:
team_name_list = ['aroknni', 'giokcon', 'cascsse', 'kussni', 'gioroce', 
                  'gossanon', 'koossani', 'joscni']

In [ ]:
x_train = df_age['name'].to_frame()

x_train = fasttext_xifyer_ngram(x_train)

y_train = df_age['rating.mean_age']

regr = ElasticNetCV(l1_ratio = [0.01, 0.05, .1, 0.2, .5, .7, .9, .95, .99, 1],
                      n_alphas = 250,
                      max_iter = 10000,
                      cv = len(x_train),
                      selection = 'random', 
                      random_state=17042020,)                                   # Set the hyperparameters of the ElasticNet CV


regr.fit(x_train, y_train)                                                    # Fit the model on the train set



fnn_model = fnn_maker(x_train, y_train, 0, 0, 300, 0.5)

for name in team_name_list:
  name_lexical = name
  name = fasttext_xifyer_ngram(pd.DataFrame([name]))
  print('ElasticNet age of {}: {}'.format(name_lexical, regr.predict(name)[0]))
  print('Neural Net age of {}: {}'.format(name_lexical, fnn_model.predict(name)[0][0]))

ElasticNet age of aroknni: 17.182334875945358
Neural Net age of aroknni: 6.0777058601379395
ElasticNet age of giokcon: 8.581746023010616
Neural Net age of giokcon: 4.2898945808410645
ElasticNet age of cascsse: -9.36396418604663
Neural Net age of cascsse: 1.92529296875
ElasticNet age of kussni: 8.9247526133524
Neural Net age of kussni: 10.295777320861816
ElasticNet age of gioroce: 0.597054816773138
Neural Net age of gioroce: 0.6133273243904114
ElasticNet age of gossanon: 5.495369272243359
Neural Net age of gossanon: 7.745650291442871
ElasticNet age of koossani: 28.821019089757726
Neural Net age of koossani: 20.26182746887207
ElasticNet age of joscni: 7.719746242444181
Neural Net age of joscni: 5.553224563598633


In [ ]:
x_train = df_gender['name'].to_frame()

x_train = fasttext_xifyer_ngram(x_train)

y_train = df_gender['rating.mean_gender']

regr = ElasticNetCV(l1_ratio = [0.01, 0.05, .1, 0.2, .5, .7, .9, .95, .99, 1],
                      n_alphas = 250,
                      max_iter = 10000,
                      cv = len(x_train),
                      selection = 'random', 
                      random_state=17042020,)                                   # Set the hyperparameters of the ElasticNet CV


regr.fit(x_train, y_train)                                                    # Fit the model on the train set

fnn_model = fnn_maker(x_train, y_train, 0, 0, 512, 0.3)

for name in team_name_list:
  name_lexical = name
  name = fasttext_xifyer_ngram(pd.DataFrame([name]))
  print('ElasticNet gender of {}: {}'.format(name_lexical, regr.predict(name)[0]))
  print('Neural Net gender of {}: {}'.format(name_lexical, fnn_model.predict(name)[0][0]))

ElasticNet gender of aroknni: -4.721449863090152
Neural Net gender of aroknni: -8.28087043762207
ElasticNet gender of giokcon: -32.141800057826764
Neural Net gender of giokcon: -30.81196403503418
ElasticNet gender of cascsse: -2.163373622357939
Neural Net gender of cascsse: -13.624003410339355
ElasticNet gender of kussni: -22.892136259319855
Neural Net gender of kussni: -34.57113265991211
ElasticNet gender of gioroce: -16.711027865954097
Neural Net gender of gioroce: -10.085737228393555
ElasticNet gender of gossanon: -1.4955818490305086
Neural Net gender of gossanon: -3.39229154586792
ElasticNet gender of koossani: -50.44790124601548
Neural Net gender of koossani: -53.07509231567383
ElasticNet gender of joscni: -13.905265246171561
Neural Net gender of joscni: -10.771788597106934


In [ ]:
x_train = df_polarity['name'].to_frame()

x_train = fasttext_xifyer_ngram(x_train)

y_train = df_polarity['rating.mean_valence']

regr = ElasticNetCV(l1_ratio = [0.01, 0.05, .1, 0.2, .5, .7, .9, .95, .99, 1],
                      n_alphas = 250,
                      max_iter = 10000,
                      cv = len(x_train),
                      selection = 'random', 
                      random_state=17042020,)                                   # Set the hyperparameters of the ElasticNet CV


regr.fit(x_train, y_train)                                                    # Fit the model on the train set

fnn_model = fnn_maker(x_train, y_train, 0, 0, 512, 0.5)

for name in team_name_list:
  name_lexical = name
  name = fasttext_xifyer_ngram(pd.DataFrame([name]))
  print('ElasticNet polarity of {}: {}'.format(name_lexical, regr.predict(name)[0]))
  print('Neural Net polarity of {}: {}'.format(name_lexical, fnn_model.predict(name)[0][0]))

ElasticNet polarity of aroknni: -1.128005619394708
Neural Net polarity of aroknni: -8.49339771270752
ElasticNet polarity of giokcon: -17.670786717807758
Neural Net polarity of giokcon: -7.082479953765869
ElasticNet polarity of cascsse: 8.865310101500237
Neural Net polarity of cascsse: 4.159171104431152
ElasticNet polarity of kussni: -33.33449282634761
Neural Net polarity of kussni: -16.897737503051758
ElasticNet polarity of gioroce: -6.893888263026655
Neural Net polarity of gioroce: -6.848810195922852
ElasticNet polarity of gossanon: 5.697306457469919
Neural Net polarity of gossanon: -3.9218926429748535
ElasticNet polarity of koossani: -27.22956290678306
Neural Net polarity of koossani: -22.332136154174805
ElasticNet polarity of joscni: -10.620442939731301
Neural Net polarity of joscni: -3.4196674823760986
